In [87]:
import os
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

GPT_MODEL = "gpt-3.5-turbo"
client = OpenAI()

In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [138]:
# Load tools from ai/tools.json
def load_tools():
    with open("ai/tools.json", "r") as f:
        tools = json.load(f)
    return tools

tools = load_tools()
tools

[{'type': 'function',
  'function': {'name': 'read_file',
   'description': 'Read the contents of a file',
   'parameters': {'type': 'object',
    'properties': {'path': {'type': 'string',
      'description': 'The path to the file to read'}},
    'required': ['path']}}},
 {'type': 'function',
  'function': {'name': 'write_file',
   'description': 'Write the contents of a file',
   'parameters': {'type': 'object',
    'properties': {'path': {'type': 'string',
      'description': 'The path to the file to write'},
     'content': {'type': 'string',
      'description': 'The content to write to the file'}},
    'required': ['path', 'content']}}},
 {'type': 'function',
  'function': {'name': 'delete_file',
   'description': 'Delete a file',
   'parameters': {'type': 'object',
    'properties': {'path': {'type': 'string',
      'description': 'The path to the file to delete'}},
    'required': ['path']}}},
 {'type': 'function',
  'function': {'name': 'list_directory',
   'description': 'Li

In [142]:
#Load functions
def read_file(tool):
    file = tool[0]
    try:
        with open(file, "r") as f:
            return f.read()
    except Exception as e:
        return e
    
def write_file(tool):
    file = tool[0]
    content = tool[1]
    try:
        with open(file, "w") as f:
            f.write(content)
        return "File written successfully"
    except Exception as e:
        return e
    
def delete_file(tool):
    file = tool[0]
    try:
        os.remove(file)
        return "File deleted successfully"
    except Exception as e:
        return e
    
def list_directory(tool):
    directory = tool[0]
    if len(tool) > 1:
        recursive = tool[1]
    else:
        recursive = True

    try:
        if(recursive):
            all_files = []
        
            # Walk through the directory tree
            for root, dirs, files in os.walk(directory):
                # Add the files to the list with their full paths
                for file in files:
                    all_files.append(os.path.join(root, file))
                # Add the directories to the list with their full paths
                for dir in dirs:
                    all_files.append(os.path.join(root, dir))
            
            return all_files
        else:
            return os.listdir(directory)
    except Exception as e:
        return e
    

def create_directory(tool):
    directory = tool[0]
    try:
        os.mkdir(directory)
        return "Directory created successfully"
    except Exception as e:
        return e
    


In [140]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Get familiar with the entire project. "})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_na630O5m8vT7pqKo70mgYqbu', function=Function(arguments='{"path":"./"}', name='list_directory'), type='function')])

In [67]:
def execute_tool(function, args=None):
    try:
        response = eval(function)(args)
        return response
    except Exception as e:
        return e


In [143]:
#Execute tool calls
tool_calls = assistant_message.tool_calls
messages = []
if tool_calls:
    for tool_call in tool_calls:
        tool_call_id = tool_call.id
        tool_function_name = tool_call.function.name
        tool_args = tool_call.function.arguments
        # convert tool_args dictionary to array
        tool_args = [eval(tool_args)[key] for key in eval(tool_args)]

        # Execute tool function
        tool_output = None
    
        tool_output = execute_tool(tool_function_name, tool_args)
  

        messages.append({"role": "tool", "tool_call_id": tool_call.id, "name": tool_function_name, "output": tool_output})
messages

[{'role': 'tool',
  'tool_call_id': 'call_na630O5m8vT7pqKo70mgYqbu',
  'name': 'list_directory',
  'output': ['./.hintrc',
   './authenticate.py',
   './log_db.py',
   './openai_testing.ipynb',
   './project.py',
   './secrets.txt',
   './.git',
   './ai',
   './instance',
   './projects',
   './screenshots',
   './static',
   './templates',
   './__pycache__',
   './.git\\COMMIT_EDITMSG',
   './.git\\config',
   './.git\\description',
   './.git\\HEAD',
   './.git\\index',
   './.git\\hooks',
   './.git\\info',
   './.git\\logs',
   './.git\\objects',
   './.git\\refs',
   './.git\\hooks\\applypatch-msg.sample',
   './.git\\hooks\\commit-msg.sample',
   './.git\\hooks\\fsmonitor-watchman.sample',
   './.git\\hooks\\post-update.sample',
   './.git\\hooks\\pre-applypatch.sample',
   './.git\\hooks\\pre-commit.sample',
   './.git\\hooks\\pre-merge-commit.sample',
   './.git\\hooks\\pre-push.sample',
   './.git\\hooks\\pre-rebase.sample',
   './.git\\hooks\\pre-receive.sample',
   './.git